In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from numpy import transpose as t
from numpy.linalg import inv
import math
from scipy.special import gamma
from scipy.special import loggamma
from numpy import log

In [2]:
data = pd.read_csv("data.csv")

In [3]:
data.head()

,PC1,PC2,Claim_price,Auction_count,Hammer_price,Final_First_auction_data,ad_si_0,ad_si_1,ad_si_2,ad_si_3,Appr_0,Appr_1,Appr_2,서울,transformed_Hammer_price,skyscraper
0,2.823222,-1.435719,1766037301,2,760000000,35,0,1,0,0,0,0,1,0,175.394072,30
1,1.278944,-2.646606,152946867,2,971889999,35,0,1,0,0,0,0,1,0,182.985785,51
2,-0.530059,2.631658,11326510,3,93399999,77,0,0,0,1,0,0,1,0,121.888773,30
3,-0.706458,0.081095,183581724,2,256899000,105,0,0,0,1,0,0,1,0,145.387692,30
4,-0.533846,0.986816,45887671,2,158660000,28,0,0,0,1,0,0,1,0,133.700341,30


In [4]:
X = data.drop(['Hammer_price','transformed_Hammer_price'],axis=1)
y = data['transformed_Hammer_price']

In [5]:
#scaling
for i in X.columns:
    m = np.mean(X[i])
    s = np.std(X[i])
    X[i] = X[i].apply(lambda x : (x-m)/s)

In [6]:
class Model:
    
    def z_function(self,data,category_index):
        p = data.shape[1]
        category = [i for i in range(p)]
        Ncategory = []
        for i in category_index:
            Ncategory.append(i)
            for j in i:
                category.remove(j)
        x = [[i] for i in category]
        for i in Ncategory:
            x.append(i)
        result = []
        cnt=0
        for i in range(len(x)): 
            count = len(list(combinations(x,i+1))) 
            for j in range(count):   
                z = [0]*p
                for k in range(i+1):
                    if len(list(combinations(x,i+1))[j][k]) == 1: #(1,2)
                        z[list(combinations(x,i+1))[j][k][0]] = 1
                    else:
                        for m in range(len(list(combinations(x,i+1))[j][k])):#(1,(2,3,4))
                            z[list(combinations(x,i+1))[j][k][m]] = 1
                result.append(z)
        return result
    
    def sig(self,X,y):
        X = np.array(X)
        n = X.shape[0]
        yhat = X@inv(t(X)@X)@t(X)@y
        res = y-yhat
        return sum(res**2)/n
        
    
    def posterior(self,X,y,category_index,g):
        nu0 = 1 
        z = Model.z_function(self,X,category_index)
        y = np.array(y)
        l = []
        for i in z:
            i = np.array(i)
            Xz = X.iloc[:,np.where(i==1)[0]]
            n,p = Xz.shape
            Xz = np.array(Xz)
            sig0 = Model.sig(self,Xz,y)
            ssr = t(y)@(np.eye(n)-g/(g+1)*Xz@inv(t(Xz)@Xz)@t(Xz))@y
            loglikelihood = (-n/2)*log(np.pi)+loggamma((nu0+n)/2)-loggamma(nu0/2)-p/2*log(1+g)+nu0/2*log(nu0*sig0)-(nu0+n)/2*log(nu0*sig0+ssr)
            l.append(loglikelihood)
        l = l/sum(l)
        return l

In [7]:
m = Model()

In [8]:
z = m.z_function(X,[[6,7,8,9],[10,11,12]])

In [9]:
posterior = m.posterior(X,y,[[6,7,8,9],[10,11,12]],X.shape[0])

In [10]:
result = pd.DataFrame()
result['model'] = z
result['posterior'] = posterior

In [11]:
result.head()

,model,posterior
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001953
1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001954
2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001955
3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001955
4,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0.001955


In [12]:
np.argmax(posterior)

509

In [13]:
result.iloc[np.argmax(posterior),:]

model        [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
posterior                                    0.00196073
Name: 509, dtype: object